<a href="https://colab.research.google.com/github/lailahach/Classification-binaire-avec-un-r-seau-de-neurones/blob/main/TP11_TP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# =========================
# MLP XOR en TensorFlow
# =========================
import os, random
import numpy as np
import tensorflow as tf

# Reproductibilité
seed = 42
os.environ["PYTHONHASHSEED"] = str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

# Données XOR
X = np.array([[0.,0.],
              [0.,1.],
              [1.,0.],
              [1.,1.]], dtype=np.float32)
y = np.array([[0.],
              [1.],
              [1.],
              [0.]], dtype=np.float32)

# Modèle 2-4-1 avec sigmoïde
model = tf.keras.Sequential([
    tf.keras.layers.Dense(4, activation="sigmoid", input_shape=(2,)),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

# --- Assertions structure (dimensions des poids) ---
# Dense #1 : kernel (in_features=2, units=4), bias (4,)
k1, b1 = model.layers[0].weights
assert tuple(k1.shape) == (2, 4), f"Poids couche cachée: attendu (2,4) obtenu {k1.shape}"
assert tuple(b1.shape) == (4,),   f"Biais couche cachée: attendu (4,) obtenu {b1.shape}"

# Dense #2 : kernel (in_features=4, units=1), bias (1,)
k2, b2 = model.layers[1].weights
assert tuple(k2.shape) == (4, 1), f"Poids couche sortie: attendu (4,1) obtenu {k2.shape}"
assert tuple(b2.shape) == (1,),   f"Biais couche sortie: attendu (1,) obtenu {b2.shape}"

# Optimiseur & perte (MSE)
optimizer = tf.keras.optimizers.SGD(learning_rate=1.0)  # XOR converge vite avec LR élevé
loss_fn   = tf.keras.losses.MeanSquaredError()

# Boucle d'entraînement manuelle (rétroprop + descente de gradient)
epochs = 5000
loss_history = []

# Perte initiale (pour assertion d'amélioration)
with tf.GradientTape() as tape:
    y_hat0 = model(X, training=True)
    loss0 = loss_fn(y, y_hat0)
loss_history.append(float(loss0.numpy()))

for epoch in range(1, epochs + 1):
    with tf.GradientTape() as tape:
        y_pred = model(X, training=True)
        loss = loss_fn(y, y_pred)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    if epoch % 500 == 0:
        loss_history.append(float(loss.numpy()))

# Évaluations
y_prob = model(X, training=False).numpy()
y_pred_bin = (y_prob >= 0.5).astype(np.float32)

# --- Assertions comportement ---
# 1) Sorties dans [0,1] (sigmoïde)
assert np.all(y_prob >= 0.0) and np.all(y_prob <= 1.0), "Les probabilités doivent être dans [0,1]."

# 2) Amélioration de la perte
assert loss_history[-1] < loss_history[0], f"La perte n'a pas diminué: {loss_history[0]:.4f} -> {loss_history[-1]:.4f}"

# 3) Forme de sortie (4,1)
assert y_prob.shape == (4,1), f"Sortie attendue (4,1), obtenu {y_prob.shape}"

print("Perte initiale:", loss_history[0])
print("Perte finale approx.:", loss_history[-1])
print("Probabilités:", np.round(y_prob, 4).reshape(-1))
print("Prédictions binaires:", y_pred_bin.reshape(-1))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Perte initiale: 0.31446683406829834
Perte finale approx.: 0.0004981525707989931
Probabilités: [0.0242 0.9794 0.9765 0.0207]
Prédictions binaires: [0. 1. 1. 0.]


In [2]:
# =========================
# MLP XOR en PyTorch
# =========================
import os, random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Reproductibilité
seed = 42
os.environ["PYTHONHASHSEED"] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Données XOR
X = torch.tensor([[0.,0.],
                  [0.,1.],
                  [1.,0.],
                  [1.,1.]], dtype=torch.float32, device=device)
y = torch.tensor([[0.],
                  [1.],
                  [1.],
                  [0.]], dtype=torch.float32, device=device)

# Modèle 2-4-1 avec sigmoïde
model = nn.Sequential(
    nn.Linear(2, 4),   # (out=4, in=2)
    nn.Sigmoid(),
    nn.Linear(4, 1),   # (out=1, in=4)
    nn.Sigmoid()
).to(device)

# --- Assertions structure (dimensions des poids) ---
l1 = model[0]
l2 = model[2]
# nn.Linear.weight a la forme (out_features, in_features)
assert tuple(l1.weight.shape) == (4, 2), f"Poids couche cachée: attendu (4,2) obtenu {tuple(l1.weight.shape)}"
assert tuple(l1.bias.shape)   == (4,),   f"Biais couche cachée: attendu (4,) obtenu {tuple(l1.bias.shape)}"
assert tuple(l2.weight.shape) == (1, 4), f"Poids couche sortie: attendu (1,4) obtenu {tuple(l2.weight.shape)}"
assert tuple(l2.bias.shape)   == (1,),   f"Biais couche sortie: attendu (1,) obtenu {tuple(l2.bias.shape)}"

# Perte & optimiseur
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=1.0)

# Perte initiale
with torch.no_grad():
    y_hat0 = model(X)
    loss0 = criterion(y_hat0, y).item()

epochs = 5000
loss_last = loss0
for epoch in range(1, epochs + 1):
    optimizer.zero_grad()
    y_pred = model(X)
    loss = criterion(y_pred, y)
    loss.backward()
    optimizer.step()
    if epoch % 500 == 0:
        loss_last = loss.item()

# Évaluations
with torch.no_grad():
    y_prob = model(X).cpu().numpy()
    y_bin  = (y_prob >= 0.5).astype(np.float32)

# --- Assertions comportement ---
# 1) Sorties dans [0,1]
assert np.all(y_prob >= 0.0) and np.all(y_prob <= 1.0), "Les probabilités doivent être dans [0,1]."

# 2) Amélioration de la perte
assert loss_last < loss0, f"La perte n'a pas diminué: {loss0:.4f} -> {loss_last:.4f}"

# 3) Forme de sortie (4,1)
assert y_prob.shape == (4,1), f"Sortie attendue (4,1), obtenu {y_prob.shape}"

print("Device:", device)
print("Perte initiale:", round(loss0, 6))
print("Perte finale approx.:", round(loss_last, 6))
print("Probabilités:", np.round(y_prob.reshape(-1), 4))
print("Prédictions binaires:", y_bin.reshape(-1))


Device: cpu
Perte initiale: 0.28656
Perte finale approx.: 0.000561
Probabilités: [0.0218 0.9737 0.9792 0.0254]
Prédictions binaires: [0. 1. 1. 0.]
